In [31]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)

In [2]:
bnf_pandas = pd.read_csv("../data/indices/NIFTY BANK.csv")

In [3]:
# symbol = 'midcp'
# symbol = 'nifty'
# symbol = 'fnf'
symbol = 'bnf'
# symbol = 'sensex'
# symbol = 'bankex'

if symbol == 'bnf' or symbol == 'bankex':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.0001
elif symbol == 'nifty' or symbol == 'fnf':
    LEVERAGE_ = 7
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.0002
elif symbol == 'midcp':
    LEVERAGE_ = 8
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.0005
elif symbol == 'sensex':
    LEVERAGE_ = 8
    LOT_SIZE_ = 10
    SLIPPAGE_ = 0.0001

PORTFOLIO = 1000000
print(LEVERAGE_)

5


In [4]:
bnf_pandas.head()

,datetime,o,h,l,c
0,2017-01-02T09:15:00.000000,18242.3000,18248.2000,18175.9000,18181.2000
1,2017-01-02T09:16:00.000000,18181.8500,18194.7000,18179.9500,18184.4500
2,2017-01-02T09:17:00.000000,18184.9500,18189.2500,18133.8000,18133.8000
3,2017-01-02T09:18:00.000000,18135.1000,18141.5500,18118.5500,18138.9500
4,2017-01-02T09:19:00.000000,18138.9500,18142.5500,18120.4500,18124.3000


In [5]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas

In [6]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [7]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

In [8]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )

In [10]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [9]:
def generate_stats_options(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["Opt ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["Opt ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["Opt ROI%"] > 0]["Opt ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["Opt ROI%"] < 0]["Opt ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["Opt ROI%"].cumsum() - year_trades["Opt ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["Opt ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["Opt ROI%"] > 0]["Opt ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["Opt ROI%"] < 0]["Opt ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["Opt ROI%"].cumsum() - combined_df_sorted["Opt ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [18]:
from expiries import dict_expiries

In [21]:
TF_ = '15m'
TF_int = 15
OPT_SLIPPAGE_ = 0.01

In [ ]:
bnf2 = resample(bnf, TF_)
bnf_pandas = bnf2.to_pandas()
bnf_pandas.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'}, inplace=True)

In [17]:
def get_monthly_expiry(entry_time, index_expiries):
    """Get the nearest monthly expiry based on entry time."""
    expiry_df = pd.DataFrame({"expiry": index_expiries})
    
    # Convert date back to datetime to use .dt accessor
    expiry_df["expiry"] = pd.to_datetime(expiry_df["expiry"])

    expiry_df["year_month"] = expiry_df["expiry"].dt.to_period("M")

    # Extract the last expiry for each month
    monthly_expiries = expiry_df.groupby("year_month")["expiry"].max().values

    # Get the next expiry after entry_time
    next_expiry = next((exp for exp in monthly_expiries if exp >= entry_time), None)

    return pd.Timestamp(next_expiry).date()

In [13]:
def ma_crossover_logic(df, fast_ma, slow_ma):
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    
    fast_ema = df['close'].rolling(fast_ma).mean()
    slow_ema = df['close'].rolling(slow_ma).mean()
    df['fast_ma'] = fast_ema
    df['slow_ma'] = slow_ema
    
    df['signal'] = 0  # Default to no signal
    df.loc[fast_ema > slow_ema, 'signal'] = 1   # Long Signal
    df.loc[fast_ema < slow_ema, 'signal'] = -1  # Short Signal
    
    return df

In [14]:
def execute(df, sl_pct, n, portfolio=100000, leverage=1, lot_size=1, slippage=0.0001, rpt = 1):
    
    trade_book = []
    in_trade_long = False
    in_trade_short = False
    signal_initial_sl_long = 0
    signal_initial_sl_short = 0
    cumulative_roi = 0
    max_drawdown = 0
    peak_roi = 0

    # print(df.tail(50).to_string())

    for i in range(1, len(df)):
        points = 0
        
        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]

        previous_candle_open = df.iloc[i-1]["open"]
        previous_candle_high = df.iloc[i-1]["high"]
        previous_candle_low = df.iloc[i-1]["low"]
        previous_candle_close = df.iloc[i-1]["close"]

        if not in_trade_long:
            if (df.iloc[i-1]['signal'] == 1) and current_candle_high >= previous_candle_high and current_candle_open <= previous_candle_high:
                # Entry Triggered
                entry_price_long = previous_candle_high
                initial_sl_long = entry_price_long * (1 - (sl_pct / 100))
                # initial_sl_long = df.iloc[max(0, i-n):i]['low'].min()
                signal_generation_time_long = df.iloc[i-1]['datetime']
                entry_time_long = df.iloc[i]['datetime']
                in_trade_long = True

        if in_trade_long:

            if current_candle_low <= initial_sl_long:
                if current_candle_open > initial_sl_long:
                    # Initial SL Hit
                    in_trade_long = False
                    exit_price_long = initial_sl_long
                    exit_time_long = df.iloc[i]['datetime']
                    points = exit_price_long - entry_price_long
                    remarks = 'ISL Hit'
                elif current_candle_open < initial_sl_long  and (df.iloc[i]['datetime'] != entry_time_long):
                    # Gap Open SL
                    in_trade_long = False
                    exit_price_long = current_candle_open
                    exit_time_long = df.iloc[i]['datetime']
                    points = exit_price_long - entry_price_long
                    remarks = 'Gap SL Hit'
                else:
                    # Initial SL Hit
                    in_trade_long = False
                    exit_price_long = initial_sl_long
                    exit_time_long = df.iloc[i]['datetime']
                    points = exit_price_long - entry_price_long
                    remarks = 'ISL Hit'
                    
            elif df.iloc[i-1]['signal'] == -1 and current_candle_low <= previous_candle_low and previous_candle_low > initial_sl_long:
                # MA Cross in Opposite Direction
                if current_candle_open >= previous_candle_low:
                    in_trade_long = False
                    exit_price_long = previous_candle_low
                    exit_time_long = df.iloc[i]['datetime']
                    points = exit_price_long - entry_price_long
                    remarks = 'MA Cross Opp'
                else:
                    if current_candle_high >= previous_candle_low:
                        in_trade_long = False
                        exit_price_long = previous_candle_low
                        exit_time_long = df.iloc[i]['datetime']
                        points = exit_price_long - entry_price_long
                        remarks = 'MA Cross Opp'
                    else:
                        in_trade_long = False
                        exit_price_long = current_candle_close
                        exit_time_long = df.iloc[i]['datetime']
                        points = exit_price_long - entry_price_long
                        remarks = 'MA Cross Opp W Gap Exit'

            if not in_trade_long and points:
                # Exit Found
                # qty = int(round((portfolio * leverage / entry_price_long) / lot_size)) * lot_size
                qty = int(round((portfolio * rpt / 100) / abs(entry_price_long - initial_sl_long)) / lot_size) * lot_size
                slippage_ = slippage * (entry_price_long + exit_price_long)
                final_points = points - slippage_
                pnl = final_points * qty
                roi = (pnl / portfolio) * 100
                
                trade_book.append({
                    "Trade Type": "LONG",
                    "Entry Time": entry_time_long,
                    "Entry Price": entry_price_long,
                    "Initial SL": initial_sl_long,
                    "Exit Time": exit_time_long,
                    "Exit Price": exit_price_long,
                    "Points Captured": points,
                    "Slippage": slippage_,
                    "Qty": qty,
                    "Final Points": final_points,
                    "PnL": pnl,
                    "ROI%": roi,
                    "Trade Duration": exit_time_long - entry_time_long,
                    "Remarks": remarks,
                })

                remarks = ""
                points = 0

        if not in_trade_short:
            if (df.iloc[i-1]['signal'] == -1) and current_candle_low <= previous_candle_low and current_candle_open >= previous_candle_low:
                # Entry Triggered for Short
                entry_price_short = previous_candle_low
                initial_sl_short = entry_price_short * (1 + (sl_pct / 100))
                # initial_sl_short = df.iloc[max(0, i-n):i]['high'].max()
                entry_time_short = df.iloc[i]['datetime']
                in_trade_short = True

        if in_trade_short:
            
            if current_candle_high >= initial_sl_short:
                if current_candle_open < initial_sl_short:
                    # Initial SL Hit for Short
                    in_trade_short = False
                    exit_price_short = initial_sl_short
                    exit_time_short = df.iloc[i]['datetime']
                    points = entry_price_short - exit_price_short
                    remarks = 'ISL Hit'
                elif current_candle_open < initial_sl_short and (df.iloc[i]['datetime'] != entry_time_short):
                    # Initial SL Hit for Short
                    in_trade_short = False
                    exit_price_short = current_candle_open
                    exit_time_short = df.iloc[i]['datetime']
                    points = entry_price_short - exit_price_short
                    remarks = 'Gap SL Hit'
                else:
                    # Initial SL Hit for Short
                    in_trade_short = False
                    exit_price_short = initial_sl_short
                    exit_time_short = df.iloc[i]['datetime']
                    points = entry_price_short - exit_price_short
                    remarks = 'ISL Hit'
                    
            elif df.iloc[i-1]['signal'] == 1 and current_candle_high >= previous_candle_high and previous_candle_high < initial_sl_short:
                # MA Cross in Opposite Direction for Short
                if current_candle_open <= previous_candle_high:
                    in_trade_short = False
                    exit_price_short = previous_candle_high
                    exit_time_short = df.iloc[i]['datetime']
                    points = entry_price_short - exit_price_short
                    remarks = 'MA Cross Opp'
                else:
                    if current_candle_low <= previous_candle_high:
                        in_trade_short = False
                        exit_price_short = previous_candle_high
                        exit_time_short = df.iloc[i]['datetime']
                        points = entry_price_short - exit_price_short
                        remarks = 'MA Cross Opp'
                    else:
                        in_trade_short = False
                        exit_price_short = current_candle_close
                        exit_time_short = df.iloc[i]['datetime']
                        points = entry_price_short - exit_price_short
                        remarks = 'MA Cross Opp W Gap Exit'

            if not in_trade_short and points:
                # Exit Found
                # qty = int(round((portfolio * leverage / entry_price_short) / lot_size)) * lot_size
                qty = int(round((portfolio * rpt / 100) / abs(entry_price_short - initial_sl_short)) / lot_size) * lot_size
                slippage_ = slippage * (entry_price_short + exit_price_short)
                final_points = points - slippage_
                pnl = final_points * qty
                roi = (pnl / portfolio) * 100

                trade_book.append({
                    "Trade Type": "SHORT",
                    "Entry Time": entry_time_short,
                    "Entry Price": entry_price_short,
                    "Initial SL": initial_sl_short,
                    "Exit Time": exit_time_short,
                    "Exit Price": exit_price_short,
                    "Points Captured": points,
                    "Slippage": slippage_,
                    "Qty": qty,
                    "Final Points": final_points,
                    "PnL": pnl,
                    "ROI%": roi,
                    "Trade Duration": exit_time_short - entry_time_short,
                    "Remarks": remarks,
                })

                remarks = ""
                points = 0

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [34]:
import pandas as pd
import numpy as np
from datetime import timedelta

async def evaluate_tradebook(tradebook_df, index_symbol, expiries_dict, lot_size=30, strike_spread=100):
    results = []
    is_roll_over = False
    spot_dataframe = bnf_pandas
    for _, row in tradebook_df.iterrows():
        trade_type = row["Trade Type"]
        entry_time = pd.to_datetime(row["Entry Time"])
        exit_time = pd.to_datetime(row["Exit Time"])
        entry_price = row["Entry Price"]
        trade_no = row.index
        spot_points = row['Final Points']
        spot_entry = row['Entry Price']
        spot_exit = row['Exit Price']
        remarks = row['Remarks']

        # Determine Option Type and ATM Strike
        option_type = "C" if trade_type == "SHORT" else "P"
        atm_strike = round(entry_price / strike_spread) * strike_spread  # Rounding to nearest 100

        # Find initial expiry based on entry date
        index_expiries = expiries_dict[index_symbol]
        index_expiries = [exp.date() for exp in index_expiries]
        index_expiries.sort()
        expiry = get_monthly_expiry(entry_time, index_expiries)
        # expiry = pd.Timestamp(expiry).date()

        total_pnl = 0
        rollover_logs = []

        # print(row)
        # print("Entry Exit Expiry", entry_time, exit_time, expiry)
        # print("Nearest", min(entry_time.date(), expiry))

        # Ensure all expiries are in datetime.date format
        valid_expiries = []
        for exp in index_expiries:
            if isinstance(exp, dt.datetime):
                exp_date = exp.date()
            elif isinstance(exp, dt.date):
                exp_date = exp
            elif isinstance(exp, int):  # Convert integer timestamps (if present)
                exp_date = dt.datetime.fromtimestamp(exp).date()
                # print(f"Converted timestamp {exp} → {exp_date}")  # Debugging log
            else:
                # print(f"Skipping invalid expiry: {exp} (type: {type(exp)})")
                continue
            # print(type(entry_time.date()), type(exp_date), type(expiry))
            if entry_time.date() <= exp_date <= expiry:
                valid_expiries.append(exp_date)
        
        # print(f"Expiries between {entry_time.date()} and {expiry}: {valid_expiries}")
        
        while entry_time < exit_time:
            # print("Entry Exit Expiry", entry_time, exit_time, expiry)
            current_expiry = expiry
            # Get next expiry if needed
            if (entry_time.date() > expiry) and is_roll_over: 
                remarks='Roll-over'
                expiry = get_monthly_expiry(entry_time + timedelta(days=1), index_expiries)
                # entry_time = pd.Timestamp(expiry) + timedelta(hours=15, minutes=15)  # Start fresh at expiry close
                # print("Rolling Over, New Entry Time & Expiry : ", entry_time, expiry)
                # Try to fetch spot data for the current entry_time date
                spot_row = spot_dataframe[spot_dataframe['datetime'].dt.date == entry_time.date()]
            
                # If no data found, check the next available date
                next_date = entry_time.date() + timedelta(days=1)
                while spot_row.empty and next_date <= expiry:
                    spot_row = spot_dataframe[spot_dataframe['datetime'].dt.date == next_date]
                    next_date += timedelta(days=1)  # Move to the next day
            
                # Ensure we found a valid spot price before proceeding
                if spot_row.empty:
                    print("No spot data available even for future dates. Exiting roll-over logic.")
                else:
                    # print(f"Spot data found for date: {spot_row.iloc[0]['datetime'].date()}")
                    atm_strike = round(spot_row.iloc[TF_int]['close'] / strike_spread) * strike_spread
                    entry_date = next_date
                    
                # print(len(spot_row))
                atm_strike = round(spot_row.iloc[TF_int]['close'] / 100) * 100

            # print(entry_time, exit_time, expiry)
            # Fetch option data for current expiry
            # print(type(entry_time.date()))
            option_df = await fetch_data(
                index=index_symbol,
                expiry=expiry,
                strike=atm_strike,
                asset_class=option_type,
                start_date=entry_time.date(),
                end_date=min(exit_time.date(), expiry),
                start_time = entry_time.time(),
                end_time=(exit_time + dt.timedelta(minutes=TF_int-1)).time() if exit_time.date() <= expiry else dt.time(15, 30)
            )

            # If no data, attempt fetching from previous expiries
            while (option_df is None or option_df.is_empty()) and expiry >= entry_time.date():
                # print(f"Missing data for {index_symbol} {option_type} {atm_strike} {expiry}. Trying previous expiry...")
            
                # Find index of current expiry in the expiry list
                if expiry in index_expiries:
                    expiry_index = index_expiries.index(expiry)
                    if expiry_index > 0:
                        prev_expiry = index_expiries[expiry_index - 1]
            
                        # Stop retrying if expiry is older than entry_time.date()
                        if prev_expiry < entry_time.date():
                            # print(f"Stopped retrying as previous expiry {prev_expiry} is older than entry date {entry_time.date()}")
                            break
                        
                        # print(f"Refetching data for previous expiry: {prev_expiry}")
                        expiry = prev_expiry  # Update expiry to the previous expiry
                        
                        # Retry fetching data
                        option_df = await fetch_data(
                            index=index_symbol,
                            expiry=expiry,
                            strike=atm_strike,
                            asset_class=option_type,
                            start_date=entry_time.date(),
                            end_date=min(exit_time.date(), expiry),
                            start_time=entry_time.time(),
                            end_time=(exit_time + dt.timedelta(minutes=TF_int)).time() if exit_time.date() <= expiry else dt.time(15, 30)
                        )
                    else:
                        # print(f"No previous expiry available for {index_symbol}")
                        break
                else:
                    # print(f"Expiry {expiry} not found in index_expiries dictionary")
                    break
            
            # Convert to Pandas DataFrame if data exists
            if option_df is None:
                # print(f"Final data fetch failed for {index_symbol} {option_type} {atm_strike}")
                break  # Exit loop if no data is available
                
            elif option_df is not None and not option_df.is_empty():
                option_df = resample(option_df, '15m')
                option_df = option_df.to_pandas()
                
                entry_option_price = option_df.iloc[0]['c']
                exit_option_price = option_df.iloc[-1]['c']
                
                if not results or results[-1]["Option Entry Time"] != entry_time:
                    results.append({
                        "Trade No.": trade_no,
                        "Trade Type": trade_type,
                        "Spot Entry": spot_entry,
                        "Spot Exit": spot_exit,
                        "ATM Strike": atm_strike,
                        "Expiry": expiry,
                        "Option Type": option_type,
                        "Option Entry Time": entry_time,
                        "Option Exit Time": exit_time,
                        "Option Entry Price": entry_option_price,
                        "Option Exit Price": exit_option_price,
                        "Option Points": entry_option_price - exit_option_price,
                        "Slippages": OPT_SLIPPAGE_ * (entry_option_price + exit_option_price),
                        "Remarks": remarks,
                        "Option Final Points": (entry_option_price - exit_option_price) - (0.01 * (entry_option_price + exit_option_price)),
                        "Spot Final Points": spot_points,
                    })
                # break
                option_df = pd.DataFrame()
            
            else:
                print(f"Final data fetch failed for {index_symbol} {option_type} {atm_strike}")
                break  # Exit loop if no data is available
                
            entry_time = pd.Timestamp(current_expiry) + timedelta(days=1, hours=9, minutes=15)

            # Set a Roll-over flag also
            is_roll_over = True

        is_roll_over = False

    return pd.DataFrame(results)


In [28]:
PORTFOLIO = 50_00_000
LEVERAGE_ = 5
LOT_SIZE_ = 30
SLIPPAGE_ = 0.0001
RPT_ = 2

In [39]:
async def setup():
    sl_pct_range = [0.5, 0.75, 1, 1.25, 1.5]
    stats_dictionary = {}
    
    PORTFOLIO = 1_00_00_000
    LEVERAGE_ = 5
    LOT_SIZE_ = 30
    STRIKE_SPREAD_ = 100
    SLIPPAGE_ = 0.0001
    
    for i in range(4, 41, 2):
        for j in range(6, 81, 2):
            for sl in sl_pct_range:
                if i < j and ((j-i) <= 16):
                    variation = f'MA1 : {i}, MA2 : {j}, SL : {sl}%'
                    print(variation)
                    df_spot = ma_crossover_logic(bnf_pandas, i, j)
                    tb = execute(df_spot, sl, 1, PORTFOLIO, LEVERAGE_, LOT_SIZE_, SLIPPAGE_, 3)
                    if len(tb) > 0:
                        tb['Trade Year'] = tb['Entry Time'].dt.year
                        tb = tb.sort_values(by="Entry Time")
                        stats = generate_stats(tb, f'SPOT -> {variation}')
        
                        for overall_roi_dd_ratio, stats_df in stats.items():
                            if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
                                print('---------------------------------------------------------------------------------------------------------------------------------------------------------------------')
                                print(stats_df.to_string())
                                # print(tb.head().to_string())

                        df = tb
                        df['Entry Time'] = pd.to_datetime(df['Entry Time'])
                        df['Exit Time'] = pd.to_datetime(df['Exit Time'])

                        option_tb = await evaluate_tradebook(df, symbol, dict_expiries, LOT_SIZE_, STRIKE_SPREAD_)

                        option_tb['Opt Slippage'] = 0.01 * (option_tb['Option Entry Price'] + option_tb['Option Exit Price'])
                        option_tb['Opt Qty'] = 5000000 * 6 / option_tb['ATM Strike']
                        option_tb['Opt PnL'] = option_tb['Opt Qty'] * option_tb['Option Final Points']
                        option_tb['Opt ROI%'] = option_tb['Opt PnL'] * 100 / PORTFOLIO
                        option_tb['Trade Year'] = option_tb['Option Entry Time'].dt.year

                        stats_opt = generate_stats_options(option_tb, f'OPT -> {variation}')
                        for overall_roi_dd_ratio, stats_df in stats_opt.items():
                            if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
                                print(stats_df.to_string())
                                print('---------------------------------------------------------------------------------------------------------------------------------------------------------------------')

In [ ]:
await setup()

MA1 : 4, MA2 : 6, SL : 0.5%
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                            Variation
2017     -36.6379          773  31.0479                2.6014             -1.2401     -62.5337      -0.5859  SPOT -> MA1 : 4, MA2 : 6, SL : 0.5%
2018      11.3946          765  30.1961                3.5295             -1.5055     -70.1955       0.1623  SPOT -> MA1 : 4, MA2 : 6, SL : 0.5%
2019      94.3033          758  31.7942                3.8548             -1.6145     -86.2721       1.0931  SPOT -> MA1 : 4, MA2 : 6, SL : 0.5%
2020     278.4848          892  26.6816                7.9496             -2.4672    -108.5010       2.5667  SPOT -> MA1 : 4, MA2 : 6, SL : 0.5%
2021     124.6349          754  32.4934                4.7741             -2.0531